In [4]:
import nltk

In [25]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [19]:
#Wczytaj shakespeare-macbeth.txt (nltk.corpus.gutenberg)


In [20]:
print(macbeth)

['[', 'The', 'Tragedie', 'of', 'Macbeth', 'by', ...]


In [23]:
from nltk import sent_tokenize
import string
def preprocess_text(text):
    text = ' '.join(text)
    text = sent_tokenize(text.lower())
    text = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in text]
    return text

In [26]:
macbeth_sentences = preprocess_text(macbeth)

In [28]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

def create_ngrams(text, n):
    n_grams = ngrams(word_tokenize(text), n)
    return [' '.join(grams) for grams in n_grams]

In [29]:
macbeth_example_sentence = macbeth_sentences[102]
print(macbeth_example_sentence)
create_ngrams(macbeth_example_sentence, 3)

the earth hath bubbles  as the water ha  s  and these are of them  whither are they vanish  d 


['the earth hath',
 'earth hath bubbles',
 'hath bubbles as',
 'bubbles as the',
 'as the water',
 'the water ha',
 'water ha s',
 'ha s and',
 's and these',
 'and these are',
 'these are of',
 'are of them',
 'of them whither',
 'them whither are',
 'whither are they',
 'are they vanish',
 'they vanish d']

In [30]:
tokenized_text = [list(map(str.lower, word_tokenize(sent))) for sent in macbeth_sentences]

In [31]:
from nltk.lm.preprocessing import pad_both_ends

tokenized_sentence = tokenized_text[20]
print(tokenized_sentence)
padded_tokenized_sentence = list(pad_both_ends(tokenized_sentence, n=2))
print(padded_tokenized_sentence)

['king']
['<s>', 'king', '</s>']


In [32]:
# też dodaje znaczniki padowania
from nltk.lm.preprocessing import padded_everygram_pipeline

n=3
training_data, vocabulary = padded_everygram_pipeline(n, tokenized_text)

In [33]:
from nltk.lm import MLE
model = MLE(3)
model.fit(training_data, vocabulary)

In [34]:
print(model.vocab.lookup('king has a home in bydgoszcz'.split()))

('king', 'has', 'a', 'home', 'in', '<UNK>')


In [35]:
model.counts[['of']]['the']

27

In [36]:
# chance that the is proceeded by of
model.score('the', ['of'])

0.07988165680473373

In [37]:
print(' '.join(model.generate(13)))

capitall confess d his treasons implor d your highnesse part is to do


In [38]:
import pandas as pd
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [39]:
#Wczytaj IMDB Dataset - https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


In [40]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [41]:
remove = re.compile("[.;:!\'?,\"()\[\]]")
replace_with_space = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [remove.sub("", line.lower()) for line in reviews]
    reviews = [replace_with_space.sub(" ", line) for line in reviews]
    return reviews

In [42]:
data['review'] = preprocess_reviews(data.review)

In [43]:
data['sentiment'] = LabelBinarizer().fit_transform(data['sentiment'])

In [44]:
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


In [45]:
train_data, test_data, train_sentiments, test_sentiments = train_test_split(data.review, data.sentiment, test_size=0.2)

In [46]:
bow_vectorizer = CountVectorizer(ngram_range = (1,2))
train_data_vectorized = bow_vectorizer.fit_transform(train_data)
test_data_vectorized = bow_vectorizer.transform(test_data)

In [47]:
data.shape

(50000, 2)

In [48]:
len(bow_vectorizer.vocabulary_)

2207611

In [49]:
MNB = MultinomialNB()
MNB.fit(train_data_vectorized, train_sentiments)

MultinomialNB()

In [50]:
predictions = MNB.predict(test_data_vectorized)
accuracy_score(test_sentiments, predictions)

0.8826